**Imports**

In [ ]:
import numpy as np
import pandas as pd
from scipy import optimize
from IPython.display import display
from itertools import combinations
from tqdm.notebook import tqdm
import numdifftools as nd
import statsmodels.api as sm
from statsmodels.formula.api import ols, logit
from os import path

# Methodology

## Cost function
Here we define the negative log-likelihood function of the provided parameters, given choice data. The `params` argument contains parameters the likelihood of which is evaluated. The last parameter must be the softmax temperature ($\tau$) parameter. In `args` we provide the choice data. It consists of $K$ 2-dimensional arrays, where $K$ is equal to the number of parameters. Each array is $N \times M$, where $N$ is the number of trials (in our case, 250) and $M$ is the number of activities (in our case, 4). Importantly, like in the `params` argument, the last element of the `args` list has a special meaning. It is reserved for the choice data, encoded as a 1-hot vector for each trial. Thus, in each row of the choice data array, all entries are 0, except for one that indicates which task was chosen on that trial. The other arrays in `args` contain learning heuristics data for each trial and each activity. The order of heuristics arrays in `args` corresponds to the order of the associated parameters (i.e. heuristic weights) in `params`.

The function computes the utility of each activity on each trial ($U_{i,t}$) as a linear combination of learning heuristics ($x_i$) and their respective weights $w_i$:

$$ U_{i,t} = \sum_{i=1}^{M} w_{i} \text{x}_{i,t} $$

These utility scores are used to compute the choice probabilities for each activity:

$$ p_t(\text{choice}_i) = \frac{e^{U_{i,t}/\tau}}{\sum_{i=1}^{M}e^{U_{m,t}/\tau}} $$

This gives us a 2-dimensional array, where each of N rows corresponds to choice probabilities across all M activities. The likelihood of parameters that predict choosing some activity $i$ on trial $t$ is equal to probability of choosing that activity on that trial given by the softmax model. Thus, we can get individual trial likelihoods by boolean-indexing the array of choice probabilities by the actual choice data. Finally, we can log-transform the resulting vector of likelihoods, and sum it up to get the overall log-likelihood of the model. The function returns the negative of log-likelihood, so that the optimization algorithm look for parameter values that minimize this objective. Minimizing the negative log-likelihood is equivalent to maximizing the log-likelihood, and since logarithmic transformation is monotonic, the optimal paramter values will define the maximum-likelihood model.

## Model fitting
We define `fit` and `fit_n` methods in the `SoftmaxChoiceModel` object that will help us find optimal parameter values. We use the BFGS optimization method provided by the `scipy` library. Each individual fitting routine (`SoftmaxChoiceModel.fit()`), takes in the subject's actual choice data, his or her learning heuristics data, and an initial parameter values (i.e. `initial_guess`), performs parameter optimization and returns the optimal parameters and the resulting cost value upon convergence. Since, in high-dimensional spaces the method may be prone to local optima, it is good to repeat this optimization routine several times using different initializations. This is done by `SoftmaxChoiceModel.fit_n()` that executes the fitting routing `n` times, updates the parameters found by previous routines, if they are better. Random parameter initializations are sampled uniformly from ranges provided in the `init_dict` and by the `tau_range` upon class instantiation. In the article, we report the best-fitting parameters found in `n = 300` optimization bouts.

In [ ]:
def neg_log_likelihood(params, *args):
    coeffs = np.array(params[:-1])
    inps = np.stack(args[:-1], axis=0).astype(float)
    U = (coeffs[:, None, None] * inps).sum(axis=0)
    exponent = np.exp(U * params[-1])
    P = (exponent.T / np.sum(exponent, axis=1)).T
    logP = np.log(P[args[-1].astype(bool)])
    logL = np.sum(logP, axis=0)
    return -logL


class SoftmaxChoiceModel(object):
    def __init__(self, objective, data, init_dict, tau_range=[0, 10]):
        self.objective = objective
        self.components = init_dict.keys()
        self.init_ranges = list(init_dict.values())
        self.tau_range = tau_range
        self.data = [data.loc[:, c+'1':c+'4'].values for c in self.components]
        self.choice_data = data.loc[:, 'ch1':'ch4'].values
        self.params = None
        self.negloglik = None
      
    def fit(self, data, init_guess):           
        results = optimize.minimize(self.objective, 
                                    x0 = init_guess, 
                                    args = data)
        return results.x, results.fun

    def fit_n(self, n, progbar=False):
        data = tuple(self.data+[self.choice_data])
        if progbar: 
            iter_ = tqdm(range(n), desc='Seed', leave=False)
        else:
            iter_ = range(n)
        for i in iter_:
            init_guess = np.array(
                [np.random.uniform(l, u) for l, u in self.init_ranges+[self.tau_range]]
            )
            x, f = self.fit(data, init_guess)
            if self.negloglik is None:
                self.negloglik = f
                self.params = x
            else:
                if f < self.negloglik:
                    self.negloglik = f
                    self.params = x
    
    def get_aic(self):
        # We add 1 to the number of params to include tau
        return 2*self.negloglik + 2*(len(self.params) + 1)
    
    def get_param_csv(self):
        return ','.join(['{:.5f}'.format(p) for p in self.params])

# Fitting models for comparison
Here we fit choice data from individual subjects. For model comparisons, we derive all subsets of parameter combinations provided in the `init_dict` and perform multiple optimization bouts for each subject and each subset.

In [ ]:
def main(model_data_path, nam_data_path, nruns, save_path=''):
    df = pd.read_csv(model_data_path, index_col='sid')
    df = df.loc[df.trial.le(60+250) & df.trial.gt(60), :]
    df = df.loc[df.nam.gt(0), :]
    
    init_tau_range = [0, 10]
    init_dict = {
        'rpc':[-1, 1],
        'rlp':[-1, 1],
        'relt': [-1, 1]
    }
    
    # Set up model comparison (get paramter combinations)
    np.random.seed(1)
    var_set = list(init_dict.keys())
    subsets = []
    for nb_vars in range(1, len(var_set)+1):
        for subset in combinations(var_set, nb_vars):
            subsets.append(subset)
            
    # Collect model data
    print('Each model subset\'s results are appended to {}'.format(path.abspath(save_path)))
    first = True
    for subset in tqdm(subsets, desc='Progress'):
        comp_data = []
        model_form = ','.join(subset)
        init_dict_subset = {k: init_dict[k] for k in subset}
        for i, sdf in tqdm_notebook(df.groupby('sid'), desc='Variable set = ({})'.format(model_form), leave=False):
            model = SoftmaxChoiceModel(
                objective = neg_log_likelihood, 
                data = sdf,
                init_dict = init_dict_subset,
                tau_range = init_tau_range
            )
            model.fit_n(nruns, progbar=True)
            group, nam = sdf.iloc[0].loc[['group', 'nam']]
            comp_data.append([i, group, nam, model_form, model.get_aic(), model.get_param_csv()])
        pd.DataFrame(
            comp_data, 
            columns=['sid', 'group', 'nam', 'vars', 'aic', 'params']
        ).to_csv(save_path, mode='w' if first else 'a', header=first, index=False)
        first = False

    
main(
    model_data_path = 'data/model_data.csv',
    nam_data_path = 'data/nam_data.csv',
    nruns = 5,
    save_path = 'data/model_results/param_fits.csv'
)

# Simulate choices with fitted parameters

# Optimal choice model